In [ ]:
# import pandas as pd
# import numpy as np
# from keras.models import Sequential
# from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# import tensorflow as tf
# import matplotlib.pyplot as plt

# # 데이터 파일 로드
# normal_data = pd.read_csv('C:/Users/7-2009-1186-16/Desktop/-sampling-frequency-hz-/vibration_data/re_condition1/300rpm(diff)/Normal/resampled_normal_data_2023_12_7_17_2.csv')
# faulty_data = pd.read_csv('C:/Users/7-2009-1186-16/Desktop/-sampling-frequency-hz-/vibration_data/re_condition1/300rpm(diff)/Fault1/resampled_fault_data_2023_12_14_13_52.csv')


# # 정상 및 비정상 데이터에 레이블 할당
# normal_data['label'] = 0
# faulty_data['label'] = 1

# # 데이터 결합
# combined_data = pd.concat([normal_data, faulty_data], ignore_index=True)

# # 특성 및 레이블 선택
# X = combined_data[['motor_x', 'motor_y', 'motor_z']].values
# y = combined_data['label'].values

# # 데이터 정규화
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# # 훈련 및 테스트 데이터 분할
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

# # CNN 입력을 위해 데이터 형태 변경
# X_train = np.expand_dims(X_train, axis=2)
# X_test = np.expand_dims(X_test, axis=2)

# # CNN 모델 구축
# model = Sequential()
# model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
# model.add(MaxPooling1D(pool_size=1))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='sigmoid'))

# # 모델 컴파일 및 훈련
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# for layer in model.layers:
#     print(layer.name)

# model.fit(X_train, y_train, epochs=3, batch_size=64, validation_data=(X_test, y_test))

# # Grad-CAM 구현
# def get_grad_cam(model, data, predicted_class, layer_name):
#     # 중간 레이어의 출력을 포함하는 새로운 모델 생성
#     grad_model = tf.keras.models.Model(
#         [model.inputs], 
#         [model.get_layer(layer_name).output, model.output]
#     )

#     # GradientTape를 사용하여 그래디언트를 계산
#     with tf.GradientTape() as tape:
#         # 입력 데이터에 대한 중간 레이어와 최종 출력
#         conv_outputs, predictions = grad_model(data)
#         loss = predictions[:, predicted_class]

#     # 그래디언트 계산
#     output = conv_outputs[0]
#     grads = tape.gradient(loss, conv_outputs)[0]

#     # 가중치 계산
#     pooled_grads = tf.reduce_mean(grads, axis=(0, 1))

#     # 클래스 활성화 맵 계산
#     cam = tf.reduce_sum(tf.multiply(pooled_grads, output), axis=-1).numpy()

#     return cam

# # 테스트 데이터 샘플 사용
# sample_data = np.expand_dims(X_test[0], axis=0)
# sample_data_tensor = tf.convert_to_tensor(sample_data, dtype=tf.float32)
# predicted_class = np.argmax(model.predict(sample_data_tensor))

# # Grad-CAM 실행
# cam = get_grad_cam(model, sample_data_tensor, predicted_class, 'conv1d_5')

# # 결과 시각화
# plt.plot(cam)
# plt.show()



In [ ]:
import pandas as pd
import numpy as np
import scipy.signal
import tensorflow as tf
from tensorflow.keras.models import Model  # Keras를 TensorFlow 2.x 버전에 맞게 임포트
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# 데이터 로드 및 전처리
normal_data = pd.read_csv('C:/Users/7-2009-1186-16/Desktop/-sampling-frequency-hz-/vibration_data/re_condition1/300rpm(diff)/Normal/resampled_normal_data_2023_12_7_17_2.csv')
faulty_data = pd.read_csv('C:/Users/7-2009-1186-16/Desktop/-sampling-frequency-hz-/vibration_data/re_condition1/300rpm(diff)/Fault1/resampled_fault_data_2023_12_14_13_52.csv')


In [ ]:
normal_data['label'] = 0
faulty_data['label'] = 1

combined_data = pd.concat([normal_data, faulty_data], ignore_index=True)
X = combined_data[['motor_x', 'motor_y', 'motor_z']].values
y = combined_data['label'].values
scaler = StandardScaler()
X = scaler.fit_transform(X)

# STFT 적용 함수
def apply_stft(data, fs=100, nperseg=3):
    stft_data = []
    for sample in data:
        _, _, Zxx = scipy.signal.stft(sample, fs=fs, nperseg=nperseg)
        stft_data.append(np.abs(Zxx))
    return np.array(stft_data)

X_stft = apply_stft(X)

# STFT 결과 시각화
sample_index = 0  # 시각화하려는 샘플 인덱스 선택
stft_result = X_stft[sample_index]

# STFT 결과를 이미지로 표시
plt.figure(figsize=(10, 6))
plt.imshow(stft_result, cmap='viridis', aspect='auto', origin='lower')
plt.colorbar(label='Magnitude')
plt.title('STFT Result')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show()

# CNN 모델 정의 및 훈련
X_train, X_test, y_train, y_test = train_test_split(X_stft, y, test_size=0.2, random_state=30)
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

model = tf.keras.Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)))
model.add(MaxPooling2D(pool_size=(1, 1)))  # 윈도우 크기와 스트라이드를 (1, 1)로 설정
model.add(Flatten())
model.add(Dense(64, activation='relu'))  # Fully Connected 레이어 추가 (예시로 하나 추가)
model.add(Dense(1, activation='sigmoid'))  # 출력 레이어

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

for layer in model.layers:
    print(layer.name)

model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Grad-CAM 구현
def get_grad_cam(model, data, predicted_class, layer_name):
    grad_model = Model([model.inputs], [model.get_layer(layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(data)
        loss = predictions[:, predicted_class]
    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(pooled_grads, output), axis=-1).numpy()
    return cam

# Grad-CAM 실행 및 시각화
sample_data = np.expand_dims(X_test[0], axis=0)
sample_data_tensor = tf.convert_to_tensor(sample_data, dtype=tf.float32)
predicted_class = np.argmax(model.predict(sample_data_tensor))
cam = get_grad_cam(model, sample_data_tensor, predicted_class, 'conv2d_4')  # 'conv2d'는 레이어 이름
plt.plot(cam[0])  # cam은 2D 배열이므로 첫 번째 요소 선택
plt.show()
